In [8]:
from imutils.video import VideoStream
from imutils.video import FPS
import argparse
import imutils
import time
import cv2

In [10]:
cv2.__version__

'4.0.1'

In [11]:
# making the tracker file : 
csrt = cv2.TrackerCSRT_create()

In [12]:
# initialize the bounding box coordinates of the object we are going to track
BB = None
# initialize the FPS throughput estimator
fps = None
# initialize webcam :
videocapture = cv2.VideoCapture(0)

In [13]:
while True:
    # grab the current frame, ### then handle if we are using a VideoStream object
    frame = videocapture.read()
    frame = frame[1]
    # check to see if we have reached the end of the stream
    if frame is None:
        break
    # resize the frame (so we can process it faster) and grab the frame dimensions
    #frame = imutils.resize(frame, width=500)
    (H, W) = frame.shape[:2]
    
    # check to see if we are currently tracking an object
    if BB is not None:
        # grab the new bounding box coordinates of the object
        (success, box) = csrt.update(frame)
        # check to see if the tracking was a success
        if success:
            (x, y, w, h) = [int(v) for v in box]
            #cv2.rectangle(frame, (x, y), (x + w, y + h),(0, 255, 0), 2)
            cv2.circle(frame,(x+w//2,y+h//2),80,(0,255,0),2)
        # update the FPS counter
        fps.update()
        fps.stop()
        # initialize the set of information we'll be displaying on the frame
        info = [
            ("Tracker", "CSRT"),
            ("Success", "Yes" if success else "No"),
            ("FPS", "{:.2f}".format(fps.fps())),
        ]
        # loop over the info tuples and draw them on our frame
        for (i, (k, v)) in enumerate(info):
            text = "{}: {}".format(k, v)
            cv2.putText(frame, text, (10, H - ((i * 20) + 20)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
        # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the 's' key is selected, we are going to "select" a bounding box to track
    if key == ord("s"):
        # select the bounding box of the object we want to track (make sure you press ENTER or SPACE after selecting the ROI)
        BB = cv2.selectROI("Frame", frame, fromCenter=False, showCrosshair=True)
        #print(BB)
        
#         csrt.save("default_csrt.xml") # saves default values of the Tracker
#                                     you can rename default_csrt.xml-> custom_csrt.xml 
# #                                     and change values in it and use it load params

#         fs = cv2.FileStorage("custom_csrt.xml",cv2.FILE_STORAGE_READ)
#         fn = fs.getFirstTopLevelNode()
#         csrt.read(fn)
        
        
        # start OpenCV object tracker using the supplied bounding boxcoordinates, 
        # then start the FPS throughput estimator as well
        csrt.init(frame, BB)
        
        
        fps = FPS().start()
        
    # if the `q` key was pressed, break from the loop
    elif key == ord("q"):
        break

In [14]:
# if we are using a webcam, release the pointer
#if not args.get("video", False):
#    vs.stop()
# otherwise, release the file pointer
#else:
videocapture.release()
# close all windows
cv2.destroyAllWindows()